In [87]:
from selenium import webdriver


nyc_url = 'https://skiplagged.com/flights/NYC/2020-02-13/2020-02-16'
nola_url = 'https://skiplagged.com/flights/MSY/2020-02-13/2020-02-16'

driver = webdriver.Chrome('./chromedriver/chromedriver')

driver.get(nyc_url)
driver.implicitly_wait(100)
nyc_source = driver.page_source

driver.quit()



In [88]:
from selenium import webdriver


nyc_url = 'https://skiplagged.com/flights/NYC/2020-02-13/2020-02-16'
nola_url = 'https://skiplagged.com/flights/MSY/2020-02-13/2020-02-16'

driver = webdriver.Chrome('./chromedriver/chromedriver')

driver.get(nola_url)
driver.implicitly_wait(100)
nola_source = driver.page_source

driver.quit()


In [74]:
from bs4 import BeautifulSoup
import pandas as pd
soup = BeautifulSoup(nyc_source, 'html.parser')
soup

<html lang="en-US"><head><script async="" src="https://buttons-config.sharethis.com/js/5977ebae1b7e610012647dbf.js"></script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="//fonts.googleapis.com/css?family=Open+Sans:400,600,700,800" rel="stylesheet" type="text/css"/>
<link href="//skipcdn.net/css/16352ddf2fad6ff76228796168ea9b32.libraries_desktop.css" rel="stylesheet"/>
<link href="//skipcdn.net/css/f2bd6f5376123b967b56ab4498455926.flights_desktop.css" rel="stylesheet"/>
<meta name="google" value="notranslate"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" name="viewport"/>
<meta content="app-id=823443083" name="apple-itunes-app"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@Skiplagged" name="twitter:site"/>
<meta content="New York to anywhere, Feb 13 - Feb 16" name="twitter:title"/>
<meta content="Skiplagged is an airfare search engine for cheap flights, showing hidden-city ticket

In [129]:
from bs4 import BeautifulSoup
import pandas as pd

def get_flights_info(out_city, source):
    soup = BeautifulSoup(source, 'html.parser')
    cities = soup.findAll('h2', {'class': 'skipsy-city'})
    regions = soup.findAll('span', {'class': 'skipsy-region'})
    costs = soup.findAll('div', {'class': 'skipsy-cost'})
    
    
    out = []
    for i in range(0, len(cities)):
        try:
            cost = int(costs[i].text[1:])
        except:
            cost = 10000
        
        row = {
            "out_city": out_city,
            "in_city": camel_case_split(cities[i].text)[0],
            "region": regions[i].text,
            "cost": cost
        }
        if(cost < 2000):
            out.append(row)

    df = pd.DataFrame(out)
    df = df[['out_city', "in_city", 'region', 'cost']]
    return df

In [125]:
def camel_case_split(str): 
    words = [[str[0]]] 
  
    for c in str[1:]: 
        if words[-1][-1].islower() and c.isupper(): 
            words.append(list(c)) 
        else: 
            words[-1].append(c) 
  
    return [''.join(word) for word in words] 

In [143]:
nyc_df = get_flights_info("New York", nyc_source)
nola_df = get_flights_info("New Orleans", nola_source)

result = pd.merge(nyc_df, nola_df, on="in_city")
result.drop(columns=['region_y'])

,out_city_x,in_city,region_x,cost_x,out_city_y,cost_y
0,New York,Knoxville,Tennessee,78,New Orleans,255
1,New York,Washington,District of Columbia,108,New Orleans,98
2,New York,Rockford,Illinois,112,New Orleans,103
3,New York,Raleigh-durham,North Carolina,124,New Orleans,93
4,New York,Detroit,Michigan,127,New Orleans,93
5,New York,Boston,Massachusetts,136,New Orleans,243
6,New York,Akron,Ohio,136,New Orleans,417
7,New York,Cleveland,Ohio,136,New Orleans,122
8,New York,Pittsburgh,Pennsylvania,136,New Orleans,184
9,New York,Greensboro,North Carolina,146,New Orleans,212


In [142]:
result

,in_city,region_x,out_city_x,cost_x,out_city_y,cost_y
0,Knoxville,Tennessee,New York,78,New Orleans,255
1,Washington,District of Columbia,New York,108,New Orleans,98
2,Rockford,Illinois,New York,112,New Orleans,103
3,Raleigh-durham,North Carolina,New York,124,New Orleans,93
4,Detroit,Michigan,New York,127,New Orleans,93
5,Boston,Massachusetts,New York,136,New Orleans,243
6,Akron,Ohio,New York,136,New Orleans,417
7,Cleveland,Ohio,New York,136,New Orleans,122
8,Pittsburgh,Pennsylvania,New York,136,New Orleans,184
9,Greensboro,North Carolina,New York,146,New Orleans,212


In [144]:
result = result[['in_city', 'region_x', 'out_city_x', 'cost_x', 'out_city_y', 'cost_y']]
result['cost_avg'] = ( result['cost_x'] + result['cost_y'] ) / 2

result = result.sort_values(by='cost_avg')
result

,in_city,region_x,out_city_x,cost_x,out_city_y,cost_y,cost_avg
1,Washington,District of Columbia,New York,108,New Orleans,98,103.0
2,Rockford,Illinois,New York,112,New Orleans,103,107.5
3,Raleigh-durham,North Carolina,New York,124,New Orleans,93,108.5
4,Detroit,Michigan,New York,127,New Orleans,93,110.0
14,Cincinnati,Ohio,New York,156,New Orleans,78,117.0
10,Atlanta,Georgia,New York,148,New Orleans,88,118.0
7,Cleveland,Ohio,New York,136,New Orleans,122,129.0
15,Cincinnati,Ohio,New York,192,New Orleans,78,135.0
31,Nashville,Tennessee,New York,184,New Orleans,103,143.5
37,Houston,Texas,New York,201,New Orleans,93,147.0


In [147]:
export_csv = result.to_csv (r'./df.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

ModuleNotFoundError: No module named 'flight_scrape'